In [10]:
#SCRAPING ENVOIREMENT

In [4]:
#Importing libaries
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import itertools
import random
import threading
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import csv

## De følgende celler skal IKKE køres

In [447]:
#Importing page-links
"""
list_url = []
for page in range(1,201,1):
    url = f'https://www.boligsiden.dk/tilsalg/villa,ejerlejlighed,raekkehus,landejendom?page={page}'
    list_url.append(url)

print(len(list_url))
"""




200


In [449]:
#Retriving HTMLS
"""
list_htmls =[]
for url in list_url:
    response = requests.get(url, headers={'name': 'Mathias Rasmussen', 'email': 'kbd655@alumni.ku.dk'})
    list_htmls.append(response.text)
    time.sleep(0.5)

"""
    

In [451]:
# Finding links on appartments
"""
links = []
for html in list_htmls:
    soup = BeautifulSoup(html, 'lxml')
    pattern = re.compile(r'href\":\"(/addresses/[^\"]+)')
    matches = pattern.findall(str(soup))

    apartments_list = list(set(matches))


    base_url = "https://www.boligsiden.dk"
    apartment_links = [base_url + link for link in apartments_list] #This list contains appartments
    links.append(apartment_links)

flattened_links = list(itertools.chain(*links))
print(len(flattened_links)) #Counting number of links
"""

9996


In [453]:
#Saving list of links to be scrapped
"""
with open('rawdata/links.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for link in flattened_links:
        writer.writerow([link])
"""


# HER STARTER SCRAPING PROCESSEN

In [6]:
#Importing relevant list of links:
flattened_links = []

with open('rawdata/links.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        flattened_links.append(row[0])  # Assuming each link is in the first column

print(len(flattened_links))

9996


In [8]:
op = webdriver.ChromeOptions()
op.add_argument('headless')
driver = webdriver.Chrome(options=op)

In [42]:
#Carefull! Runs with X-links at a time
#NANNAS VERSION
dict = {}
links_med_fejl = []

def fetch_data(link):
    driver = webdriver.Chrome(options=op)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
            
        #Finder label hvor afstand er under
        """
        try:
            school = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school = "fejl"
        #Karakter
        try:
            grades = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades = "fejl"
        #Trivsel
        try:
            trivsel = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel = "fejl"
        """
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"

        #################   NY KODE med skoler    ########################
        #Skolenavn
        #Skolenavn
        try:
            school_name_1 = soup.find('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_1 = "fejl"
        try:
            school_name_2 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_2 = "fejl"
        try:
            school_name_3 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_3 = "fejl"

        #Afstand 
        try:
            school_1 = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_1 = "fejl"
        try:
            school_2 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_2 = "fejl"
        try:
            school_3 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_3 = "fejl"

        #Karakterer
        try:
            grades_1 = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_1 = "fejl"
        try:
            grades_2 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_2 = "fejl"
        try:
            grades_3 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_3 = "fejl"

        #Trivsel
        try:
            trivsel_1 = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except: 
            trivsel_1 = "fejl"
        try:
            trivsel_2 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_2 = "fejl"
        try:
            trivsel_3 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_3 = "fejl"

        #ENDELIGE 
        
        if grades_1 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_1
            school = school_1
            grades = grades_1
            trivsel = trivsel_1
        elif grades_2 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_2
            school = school_2
            grades = grades_2
            trivsel = trivsel_2
        elif grades_3 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_3
            school = school_3
            grades = grades_3
            trivsel = trivsel_3
        else:
            school_name=school_name_1
            school = school_1 # Fallback to school_1 if all are "fejl" or "Ikke oplyst"
            grades = grades_1
            trivsel = trivsel_1



        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    list(tqdm(executor.map(fetch_data, flattened_links[9000:10000]), total=len(flattened_links[9000:10000])))


  0%|                                                   | 0/996 [00:00<?, ?it/s]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50b0-3e7a-32b8-e044-0003ba298018 - list index out of range


  0%|                                           | 2/996 [00:07<52:17,  3.16s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50c0-832d-32b8-e044-0003ba298018 - list index out of range


  1%|▍                                       | 10/996 [01:27<4:48:36, 17.56s/it]

...

 96%|███████████████████████████████████████▏ | 952/996 [28:38<00:52,  1.19s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50af-9285-32b8-e044-0003ba298018 - list index out of range
Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50c7-3af4-32b8-e044-0003ba298018 - list index out of range


 97%|███████████████████████████████████████▌ | 962/996 [30:05<02:36,  4.60s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50a8-4802-32b8-e044-0003ba298018 - list index out of range


100%|█████████████████████████████████████████| 996/996 [31:01<00:00,  1.87s/it]


In [2]:
#Sikkerhedskopi
"""
#Carefull! Runs with X-links at a time
dict = {}
links_med_fejl = []

def fetch_data(link):
    #service = Service(ChromeDriverManager().install())
    #driver = webdriver.Chrome(service=service)
    driver = webdriver.Chrome(options=op)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
        #Finder label hvor afstand er under
        try:
            school = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school = "fejl"
        #Karakter
        try:
            grades = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades = "fejl"
        #Trivsel
        try:
            trivsel = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel = "fejl"
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"


        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'Kommune_tekst': kommune_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'Kommune_tekst': kommune_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    list(tqdm(executor.map(fetch_data, flattened_links[0:50]), total=len(flattened_links)))
"""

NameError: name 'ThreadPoolExecutor' is not defined

In [44]:
#Denne celle henter data på alle links med fejl fra ovenstående celle! Kør derfor efter ovenstående.
def fetch_data(link):
    driver = webdriver.Chrome(options=op)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
    
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"

        
        #################   NY KODE med skoler    ########################
        #Skolenavn
        try:
            school_name_1 = soup.find('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_1 = "fejl"
        try:
            school_name_2 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_2 = "fejl"
        try:
            school_name_3 = soup.find('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').find_next('div', class_='text-sm font-bold text-blue-900').get_text()
        except:
            school_name_3 = "fejl"

        #Afstand 
        try:
            school_1 = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_1 = "fejl"
        try:
            school_2 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_2 = "fejl"
        try:
            school_3 = soup.find('div', string='Afstand').find_next('div', string='Afstand').find_next('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school_3 = "fejl"

        #Karakterer
        try:
            grades_1 = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_1 = "fejl"
        try:
            grades_2 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_2 = "fejl"
        try:
            grades_3 = soup.find('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades_3 = "fejl"

        #Trivsel
        try:
            trivsel_1 = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except: 
            trivsel_1 = "fejl"
        try:
            trivsel_2 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_2 = "fejl"
        try:
            trivsel_3 = soup.find('div', string='Trivsel').find_next('div', string='Trivsel').find_next('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel_3 = "fejl"

        #ENDELIGE 
        
        if grades_1 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_1
            school = school_1
            grades = grades_1
            trivsel = trivsel_1
        elif grades_2 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_2
            school = school_2
            grades = grades_2
            trivsel = trivsel_2
        elif grades_3 not in ["fejl", "Ikke oplyst"]:
            school_name=school_name_3
            school = school_3
            grades = grades_3
            trivsel = trivsel_3
        else:
            school_name=school_name_1
            school = school_1 # Fallback to school_1 if all are "fejl" or "Ikke oplyst"
            grades = grades_1
            trivsel = trivsel_1


        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Schoolname': school_name, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Kommune tekst': kommune_text, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=1) as executor:
    list(tqdm(executor.map(fetch_data, links_med_fejl), total=len(links_med_fejl)))


 12%|█████▌                                      | 2/16 [00:10<01:09,  4.99s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50c0-832d-32b8-e044-0003ba298018 - list index out of range


 50%|██████████████████████                      | 8/16 [01:54<02:10, 16.30s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50bf-54e0-32b8-e044-0003ba298018 - list index out of range


 75%|████████████████████████████████▎          | 12/16 [02:16<00:32,  8.03s/it]

Error occurred for link: https://www.boligsiden.dk/addresses/0a3f50be-3618-32b8-e044-0003ba298018 - list index out of range


100%|███████████████████████████████████████████| 16/16 [02:38<00:00,  9.93s/it]


In [376]:
#sikkerheds

"""
#Denne celle henter data på alle links med fejl fra ovenstående celle! Kør derfor efter ovenstående.
def fetch_data(link):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    try:
        driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(random.uniform(2, 3))  # Reducing sleep time slightly
        area_size = soup.find_all('span', class_='text-gray-800 whitespace-nowrap cursor-pointer border-b border-gray-400 pb-0.5 border-dotted text-sm')
        other = soup.find_all('span', class_="text-gray-800 whitespace-nowrap border-dotted text-sm")
        address = soup.find_all('div', class_='font-bold text-sm md:text-base')
        pstnr = soup.find_all('div', class_="mt-1 text-xs md:text-sm text-gray-600")
        pstnr_ny = re.findall(r'\d+', pstnr[0].get_text())  # Searching for numbers in postal-code string.
        try:
            total_address = address[0].get_text() + ',' + pstnr_ny[0]
        except:
            print('adresseproblem')
        #Nannas kode:
        try:    
            price = soup.find('h2', class_='text-2xl xl:text-3xl text-blue-900 font-bold').get_text()
        except:
            price = "fejl"
        #Finder label hvor afstand er under
        try:
            school = soup.find('div', string='Afstand').find_next_sibling('div').get_text()
        except:
            school = "fejl"
        #Karakter
        try:
            grades = soup.find('div', string='Karaktergennemsnit').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            grades = "fejl"
        #Trivsel
        try:
            trivsel = soup.find('div', string='Trivsel').find_next_sibling('div').find_next_sibling('div').get_text()
        except:
            trivsel = "fejl"
        try:
            boligtype = soup.find('span', class_='text-black text-sm pr-2').get_text()
        except:
            boligtype = "fejl"
        #energi = soup.find('svg', id='Lag_1').find('title')
        try:
            energi = soup.find('title', string=re.compile('Energimærke')).get_text()
        except:
            energi = "fejl"
        try:
            radon = soup.find('div', class_='h-7 text-sm font-bold').get_text()
        except:
            radon = "fejl"
    
        #kommune_info og liggetid med text behandling
        try:
            kommune_info = soup.find('div', class_='sec_municipality_info')
            if kommune_info:
                kommune_text = kommune_info.get_text()
        
                if 'Kommuneskatten er på ' in kommune_text:
                    kommuneskat = kommune_text.split('Kommuneskatten er på ')[1].split(' ')[0]
                else:
                    kommuneskat = "fejl"
        
                if 'grundskyldspromillen på ' in kommune_text:
                    grundskyldspromillen = kommune_text.split('grundskyldspromillen på ')[1].split(' ')[0]
                else:
                    grundskyldspromillen = "fejl"
        except:
            kommuneskat = "fejl"
            grundskyldspromillen = "fejl"
            
            
        
        try:
            liggetid = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800')
            if liggetid:
                liggetid_text = liggetid.get_text(separator=" ", strip=True).replace(' :', ':')
        except:
            liggetid_text = "fejl"


        
        try:
            if len(area_size) > 2:  # Bolig
                dict[total_address] = {'Price': price, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Grund': area_size[1].get_text(), 'Ejerudgift': area_size[2].get_text(),
                                       'Rooms': other[0].get_text(), 'Build': other[1].get_text(), 'Toilets': other[2].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
            else:  # Lejlighed
                dict[total_address] = {'Price': price, 'Distance to school': school, 'School-grades': grades, 'Trivsel': trivsel, 'Boligtype': boligtype, 'Energy-classification': energi, 'Radon': radon, 'Kommuneskat': kommuneskat, 'Grundskyldspromillen': grundskyldspromillen, 'Liggetid': liggetid_text, 'm2': area_size[0].get_text(), 'Ejerudgift': area_size[1].get_text(), 'Elevator': other[0].get_text(),
                                       'Rooms': other[1].get_text(), 'Build': other[2].get_text(), 'Toilets': other[3].get_text(),
                                       'Altan': other[4].get_text(), 'Terasse': other[5].get_text(), 'Postal code': pstnr_ny[0], 'link': link}
        except:
            dict[total_address] = {'m2': 'fejl', 'Ejerudgift': 'fejl', 'Elevator': 'fejl', 'Rooms': 'fejl', 'Build': 'fejl', 'Toilet': 'fejl', 'Altan': 'fejl', 'Terasse': 'fejl', 'Postal code': 'fejl', 'link': link}
    except Exception as e:
        print(f'Error occurred for link: {link} - {e}')
        links_med_fejl.append(link)

# Using ThreadPoolExecutor to handle multiple requests concurrently
with ThreadPoolExecutor(max_workers=1) as executor:
    list(tqdm(executor.map(fetch_data, links_med_fejl), total=len(links_med_fejl)))
""""

NameError: name 'liste_med_fejl' is not defined

## Lav først til DataFrame og efterfølgende til CSV fil

In [46]:
#Converting dict to dataframe:
DF_NAME = 'index_9000_10000' #Her kalder du din Dataframe/CSV-fil noget unikt.

df_DF_NAME = pd.DataFrame.from_dict(dict, orient='index')
pd.set_option('display.max_colwidth', None)
df_DF_NAME



,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,...,Elevator,Rooms,Build,Toilets,Altan,Terasse,Postal code,link,Grund,Toilet
"Kirkevej 34, 1. 5.,8660",998.000 kr.,Morten Børup Skolen,425 m,7.4,3.7,Ejerlejlighed,Energimærke C,Mellem,26%,"24,66‰.",...,Elevator: Nej,2 værelser,Opført 1972,1 toilet,Altan: Ja,Terrasse: Nej,8660,https://www.boligsiden.dk/addresses/0a3f50c2-311d-32b8-e044-0003ba298018,NaN,NaN
"Skovlundvej 4,7330",3.100.000 kr.,Artium Skole,"1,2 km",Ikke oplyst,Ikke oplyst,Villa,Energimærke A2020,Lav,"25,1%","20,35‰.",...,NaN,5 værelser,Opført 2025,-,NaN,NaN,7330,https://www.boligsiden.dk/addresses/2776c0a9-002c-4414-8624-903cd931adef,Grund: 941 m²,NaN
"Kløvervej 29, st.,4000",4.995.000 kr.,Hedegårdenes Skole,662 m,7.3,3.7,Ejerlejlighed,Energimærke E,Høj,"25,2%","25,69‰.",...,Elevator: Nej,6 værelser,Opført 1949,2 toiletter,Altan: Nej,Terrasse: Nej,4000,https://www.boligsiden.dk/addresses/0a3f50ab-f224-32b8-e044-0003ba298018,NaN,NaN
"Brombærhaven 50,7120",3.575.000 kr.,Hældagerskolen,"1,34 km",7.3,3.8,Villa,Energimærke C,Høj,"23,4%","27,75‰.",...,NaN,6 værelser,Opført 1999,2 toiletter,NaN,NaN,7120,https://www.boligsiden.dk/addresses/0a3f50bc-a07f-32b8-e044-0003ba298018,Grund: 890 m²,NaN
"Vestermøllevej 21,7700",2.995.000 kr.,Hanstholm Skole,"10,47 km",6.2,3.6,Villa/Fritidsbolig,Energimærke C,Lav,"25,5%","23,75‰.",...,NaN,9 værelser,Opført 1934,3 toiletter,NaN,NaN,7700,https://www.boligsiden.dk/addresses/0a3f50c5-020b-32b8-e044-0003ba298018,Grund: 984 m²,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Enghavevej 4,9530",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,fejl,fejl,https://www.boligsiden.dk/addresses/0a3f50c9-5dc4-32b8-e044-0003ba298018,NaN,fejl
"Søndergårdsparken 1,8660",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,fejl,fejl,https://www.boligsiden.dk/addresses/2734ffdb-b506-4c83-9823-d3f41e7c24a7,NaN,fejl
"Højbyvej 16,4500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,fejl,fejl,https://www.boligsiden.dk/addresses/0a3f50ae-0819-32b8-e044-0003ba298018,NaN,fejl
"Vievang Alle 2A,2610",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,fejl,fejl,https://www.boligsiden.dk/addresses/0a3f50a6-c2ac-32b8-e044-0003ba298018,NaN,fejl


In [48]:
#Converts to csv-file and save in "Rawdata"
df_DF_NAME.to_csv(f'rawdata/{DF_NAME}.csv', index=True)

